In [1]:
# Import sympy for defining symbols
import sympy as sp
# Import mpmath for increasing numerical accuracy
import mpmath as mp

from IPython.display import display

# Import InflationPy package
from inflationpy import SlowRollModel
from inflationpy import InflationFunction
from inflationpy import SlowRollEndSolver, SlowRollStartSolver

# Model

Let's define minimally connected scalar field theory with power-law potential.

We can choose formalism between metric and palatini.

In [2]:
# It is advised to define used symbols with sympy to prevent errors and for easier later use.
phi, I_phi = sp.symbols("phi I_phi", real=True)
# Define p as model free parameter and N as N-fold symbol
p, N, M_p = sp.symbols("p N M_p", real=True, positive=True)

A = InflationFunction("1")
B = InflationFunction("1")
V = InflationFunction((phi/M_p)**p)
# We must change I_V symbol to I_phi as by default it assumes that symbolI = sp.Symbol("I_phi", real=True)
I_V = InflationFunction((I_phi/M_p)**p, symbol=I_phi)
# Let's set formalism to palatini, by default formalism is metric (palatini=False)
model = SlowRollModel(A=A, B=B, V=V, I_V=I_V, palatini=True)


We have now defined one inflation model and can do calculations with it. We can see our defined functions:

In [3]:
model

A= 1
B= 1
V= (phi/M_p)**p
I_V= (I_phi/M_p)**p

We can check inspect model to check in what domains functions have defined. We can quickly check that if:
$$A(\phi) > 0 $$
$$F[A(\phi), B(\phi)] > 0$$
$$B(\phi), \  V(\phi), \ I_V(I_\phi)  \text{ domain in } \mathbb{R}$$
$$\epsilon, \  \eta \text{ domain in } \mathbb{R}$$

In some cases if function is too complicated sympy can't get results and then '-' will be shown as result.

In [4]:
model.inspect();

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# Replace parameters in model

By default inflationpy uses following symbols (in square brackest assumptions for sympy symbol):
* $\phi$ (phi)  - default symbol for scalar field [real=True]
* $I_\phi$ (I_phi)  - default symbol for invariant scalar field [real=True]
* $M_p$ (M_p) - default symbol for Planck's mass [real=True, positive=True]
* $N$ (N) - symbol for N-fold (not possible to change) [real=True, positive=True]
* $I_V$ (I_V) - symbol for invariant potential (not possible to change) [real=True]

(Invariant) Scalar field symbol can be changed only when defining InflationFunction.

Planck's mass symbol can be changed any time.

Other predefined symbols can't be changed.

## Replace symbols with other symbol or numerical value

In [5]:
print(f"Current model's Planck's mass symbol/value: {model.mp}")
display(model.mp)
print(f"Current parameter p symbol/value: {list(model.free_symbols()-{model.symbol, model.symbolI, model.mp})[0]}")
display(list(model.free_symbols()-{model.symbol, model.symbolI, model.mp})[0])

Current model's Planck's mass symbol/value: M_p


M_p

Current parameter p symbol/value: p


p

Let's assign parameter p new symbol and value for M_p. To assign new symbol/value user must insert dictionary like that:
```Python
replacement_dictionary = {old_symbol: new_symbol, ...} # Can have multiple symbols to replace
```

old_symbol - can be string or Sp.Symbol type object. Program compares symbols which to change in string form.

new_symbol - can be string or Sp.Symbol type object.

In [6]:
new_parameter_symbol = sp.Symbol('b', real=True, positive=True)
# Change p -> b and let's change M_p to 1 so scalar field is now measuerd in Planck's masses.
new_model = model({p:new_parameter_symbol, 'M_p': 1})

In [7]:
new_model

A= 1
B= 1
V= phi**b
I_V= I_phi**b

In [8]:
print(f"Current model's Planck's mass symbol/value: {new_model.mp}")
display(new_model.mp)
print(f"Current parameter p symbol/value: {list(new_model.free_symbols()-{new_model.symbol, new_model.symbolI, new_model.mp})[0]}")
display(list(new_model.free_symbols()-{new_model.symbol, new_model.symbolI, new_model.mp})[0])

Current model's Planck's mass symbol/value: 1


1

Current parameter p symbol/value: b


b

It is also possible to use longer notation as well. Let's define all free parameters.

In [9]:
new_model = model.insert_parameters({p: 2, M_p:1})
new_model

A= 1
B= 1
V= phi**2
I_V= I_phi**2

# Inflation end value

It is possible to do analytical and numerical solving.

Analytical solving uses sympy package which has it's own limits. This means that in some cases it can't solve equations or integrate. 

Numerical solving uses numpy/scipy or mpmath (for higher precision). Using numpy/scipy is faster as it can use vectorization in some cases but it is less precise (in some cases important). Mpmath with high digital precision gets slower.

SlowRollEndSolver takes in variable 'invariant' which determines if we use invariant potential for calculations. By default invariant=False.

In [10]:
# Define solver and insert InflationModel or InflationFunction of epsilon function (latter is already defined in InflationModel).
end_solver = SlowRollEndSolver(model, invariant=False)

#### Analytical solving

Sympy has two ways to solve analytical functions. One is called 'solveset' and other is 'solve'. By default it uses 'solve' as in current state it is more powerful. Results are returned in a list.

NB! Returned values in list are sp.Expr. These don't behave like InflationFunction but can be easily converted to one.

To substitute symbols use sp.subs(...) function.subs(...)
To define to numerical function use sp.lambdify(...)
More about that in https://docs.sympy.org/latest/index.html.

In [11]:
# Calculate solutions for possible scalar field end values
sym_solutions_for_end = end_solver.solve(method='solve')
# Print solutions
print("Result functions for end value:", sym_solutions_for_end)

Result functions for end value: [-sqrt(2)*M_p*p/2, sqrt(2)*M_p*p/2]


#### Numerical solving

Currently we initialized SlowRollEndSolver with 'model' which has two free parameters (p and M_p). To calculate numerically these must be defined. It is possible to define SlowRollModel where these parameters have numerical values otherwise user must give parameter values when calculating.

In [12]:
# Calculate solutions for possible scalar field end values: numpy
num_solutions_for_end_np = end_solver.nsolve_np(interval=[-2, 2], params={'p':2, 'M_p':1})
print(num_solutions_for_end_np)
# Calculate solutions for possible scalar field end values: mpmath
num_solutions_for_end_mp = end_solver.nsolve_mp(interval=[-2, 2], params={'p':2, M_p:1}, mp_kwargs={'N_points':100000})
print(num_solutions_for_end_mp)


[-1.41421356  1.41421356]
[mpf('1.414213562373095'), mpf('-1.414213562373095')]


### Increase mpmath accuracy from 15 to 30

In [13]:
print(f"Current mpmath accuracy: {mp.mp.dps}")
mp.mp.dps = 30
print(f"New mpmath accuracy: {mp.mp.dps}")

Current mpmath accuracy: 15
New mpmath accuracy: 30


In [14]:
num_solutions_for_end_mp_dps30 = end_solver.nsolve_mp(interval=[-2, 2], params={'p':2, M_p:1})
print(num_solutions_for_end_mp_dps30)


[mpf('-1.4142135623730950488016887242097'), mpf('1.4142135623730950488016887242097')]


# Inflation initial value

To calculate (invariant) scalar field initial value we use SlowRollStartSolver. This returns function of $\phi(N)$ which gives scalar field initial value for given N-fold value. To calculate this function we need to know (invariant) scalar field end value.

SlowRollStartSolver takes in variable 'invariant' which determines if we use invariant potential for calculations. By default invariant=False.

In [15]:
# Define solver and insert InflationModel or InflationFunction of epsilon function (latter is already defined in InflationModel).
initial_solver = SlowRollStartSolver(model)

#### Analytical solving

NB! Returned values in list are sp.Expr. These don't behave like InflationFunction but can be easily converted to one.

To substitute symbols use sp.subs(...) function.subs(...)
To define to numerical function use sp.lambdify(...)
More about that in https://docs.sympy.org/latest/index.html.

In [16]:
# Calculate solutions for possible scalar field end values
# Give end value calculated previously
sym_solutions_for_start = initial_solver.solve(end_value=sym_solutions_for_end[1])
# Print solutions
print("Result functions for initial value:", sym_solutions_for_start)

Result functions for initial value: [-sqrt(2)*M_p*sqrt(p)*sqrt(4*N + p)/2, sqrt(2)*M_p*sqrt(p)*sqrt(4*N + p)/2]


In [17]:
sym_solutions_for_start[1]

sqrt(2)*M_p*sqrt(p)*sqrt(4*N + p)/2

#### Numerical solving

To calculate numerically user can give maximum N-fold value (default 100). This way differential equation is solved where $\phi_\text{initial} = \phi_{\text{end}}$ sets N=0 and we solve dif. equation till N_max. Numpy and mpmath methods both return python function which takes in N-fold value.


In [18]:
# Calculate solutions for possible scalar field start values: numpy

# Let's use end_value from analytical calculation. First we must substitute free parameters:
end_value_subs = sym_solutions_for_end[1].subs({M_p:1, p:2})
num_solutions_for_start_np = initial_solver.nsolve_np(end_value=end_value_subs, N_max=100, params={"M_p":1, 'p':2})
print(f"Scalar field value at N=50: {num_solutions_for_start_np(50)}")
# Calculate solutions for possible scalar field start values: mpmath
num_solutions_for_start_mp = initial_solver.nsolve_mp(num_solutions_for_end_mp[-1], params={"M_p":1, 'p':2})
print(f"Scalar field value at N=50: {num_solutions_for_start_mp(50)}")


Scalar field value at N=50: [14.22655573]
Scalar field value at N=50: -14.2126704035518954969709298929


From the results we can see discrepancy between scipy and mpmath solutions in the 4th decimal place. In some cases this might have relevantly big effect on further calculations.

Reason for this discrepancy comes from solving numerically differential equation and mpmath can do it more precisely.

# Observational values

To calculate observational values scalar spectral index ($n_s$) and tensor-to-scalar ration ($r$) both analytical and numrical solutions can be used.

There are three possible modes:

    sympy - Result is return in analytical form. Not required to define free variables.
    numpy - Result is return in numpy array. All free variables must be defined. (default)
    mpmath - Result is return in numpy array. All free variables must be defined. More accurate than numpy.

In [19]:
# mode = sympy and result is returned as sympy expression. This returns n_s(N) function
model.calculate_ns(scalar_value=sym_solutions_for_start[1], params={"p":2, "M_p":1, "N":50})

97/101

In [20]:
# mode = sympy and result is returned as sympy expression. This returns  n_s(N) function where N=50
model.calculate_ns(scalar_value=sym_solutions_for_start[1])

-6*p/(4*N + p) + 1 + 4*(p - 1)/(4*N + p)

In [21]:
# p = Sp.Symbol('p', real=True) -  where real=True must be added.
print(f"Analytical: r = {model.calculate_r(scalar_value=sym_solutions_for_start[1], params={'p':1, 'N':50, 'M_p':1})}")

Analytical: r = 16/201


In [22]:
# numpy
# Insert single value to calcualte scalar spectral index
observational_ns_numpy = model.calculate_r_np(scalar_value=num_solutions_for_start_np(50), params={"M_p":1, "p":2})
print(observational_ns_numpy)
# It is also possible to insert list/array of values to calculate for all scalar field values
model.calculate_r_np(scalar_value=[num_solutions_for_start_np(x) for x in [50, 55, 60]], params={"M_p":1, "p":2})

[0.15810676]


array([[0.15810676],
       [0.14387387],
       [0.13199108]])

In [23]:
# mode = numpy
# Insert single value to calcualte tensor-to-scalar ratio
observational_r_mpmath = model.calculate_r_mp(scalar_value=num_solutions_for_start_np(50), params={"M_p":1, "p":2})
print(observational_r_mpmath)
# It is also possible to insert list/array of values to calculate for all scalar field values
model.calculate_r_mp(scalar_value=[str(num_solutions_for_start_np(x)[-1]) for x in [50, 55, 60]], params={"M_p":1, "p":2})

[mpf('0.158106760141315156566583428293279')]


[mpf('0.158106760141315151767157664500717'),
 mpf('0.143873870595566747262973245031731'),
 mpf('0.131991081009343255816581453007799')]